In [ ]:
import numpy as np
import MEAutility as mu
import matplotlib.pyplot as plt
from pathlib import Path
import model
from bluepyopt import ephys
import neuroplotlib as nplt

%matplotlib notebook

In [ ]:
probe_type = "planar" #'linear' #
model_names = ["hay", "hay_ais"] # "hay_ais" #"hallermann" #'hay'
model_folders = [(Path(".") / f"{model_name}_model").absolute() for model_name in model_names]

In [ ]:
# load eaps
eaps = {}
for model_name in model_names:
    output_folder = Path(f"../data/{model_name}_ecode_probe_{probe_type}") / "extracellular"
    eap = np.load(output_folder / "template.npy")
    eaps[model_name] = eap

In [ ]:
vscale = 1.5 * np.max(np.abs(list(eaps.values())))

In [ ]:
probe = model.define_electrode(probe_type=probe_type)

In [ ]:
cell = model.create(model_name=model_names[1], release=True)
sim = ephys.simulators.LFPySimulator(cell, cvode_active=True, electrode=probe)

In [ ]:
cell.freeze(param_dict={})
cell.instantiate(sim=sim)

In [ ]:
fig, ax = plt.subplots()
norm = False
for i, (model_name, eap) in enumerate(eaps.items()):
    if norm:
        vscale = 1.5
        eap_to_plot = eap / np.max(np.abs(eap), 1, keepdims=True)
    else:
        eap_to_plot = eap
    mu.plot_mea_recording(eap_to_plot, probe, colors=f"C{i}", ax=ax)
    max_channel = np.unravel_index(np.argmax(np.abs(eap)), eap.shape)[0]
    ax.plot(probe.positions[max_channel, 0], probe.positions[max_channel, 1], color=f"C{i}", marker="*")

In [ ]:
nplt.plot_detailed_neuron(cell.LFPyCell, ax=ax, color_ais="r", color_hillock="k",
                          color_axon="b", color_myelin="g", plane="xy", lw=2)

In [ ]:
cell.unfreeze(param_names=[])